In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import pathlib 
import math
import cmath
from scipy import stats
from scipy.constants import k
from copy import deepcopy
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
import random
import folium
from datetime import datetime
import time
import responses
import googlemaps
import numpy as np
import copy
from sklearn.cluster import KMeans
%matplotlib inline
import pickle

In [10]:
with open("API_key.txt", "r") as f:
    GMAP_API_KEY = f.read()
"""
This calls the Google Maps API on all pairwise combinations of a supplied list of M addresses, and computes an MxM matrix for distances and times.

Input: List of addresses

Output: Distance Matrix, Time Matrix, in seconds and in meters
"""

def getGoogleTimeDistance(addresses):
    gmaps = googlemaps.Client(key=GMAP_API_KEY)

    curDate = datetime.now()

    time_matrix = np.array([[0] * len(addresses)] * len(addresses))
    distance_matrix = np.array([[0] * len(addresses)] * len(addresses))

    # Iterate through all possible address combinations.
    for i in range(0, len(addresses)):
        for j in range(0, len(addresses)):

            # Do not calculate distance for the same address
            if i == j:
                continue
           
            # Call Google API and get response.
            result = gmaps.distance_matrix(addresses[i], addresses[j], mode='driving', units='metric',traffic_model='best_guess', departure_time=curDate)

            # Index to relevant part of response JSON File.
            curElement = result['rows'][0]['elements'][0]

            # Update matrices
            distance_matrix[i][j] = curElement['distance']['value']
            time_matrix[i][j] = curElement['duration']['value']

    return time_matrix, distance_matrix

def coords_from_address(address):
    """
    inputs: str address
    outputs: tuple[float]  result_loc
    description: converts address to into latitude and longitude
    """
    geolocator = Nominatim(user_agent = "test")
    location = geolocator.geocode(address)
    result_loc = (location.latitude, location.longitude)
    return result_loc

def write_gps_to_csv(gps_list, filename):
    df = pd.DataFrame(gps_list, columns=['Latitude', 'Longitude'])
    df.to_csv(filename, index=False)
    
#write_gps_to_csv(all_coords, "coords.csv")

In [11]:
coords = []
addrs = [
    "2701 Riverside Dr, Ottawa",
    "2515 Bank St, Ottawa",
    "30 Haxby Private, Ottawa",
    "116 Sai Crescent",
    "214 Plymouth St, Ottawa",
    "266 Lorry Greenberg Dr, Ottawa",
    "800 King Edward Ave, Ottawa",
    "2210 Bank St, Ottawa",
    "2161 Thurston Dr, Ottawa",
    "23 Ella Street, Ottawa",
    "11 Oblats Avenue, Ottawa",
    "422 Byron Avenue, Ottawa",
    "1680 Fisher Avenue, Ottawa",
    "825 Cahill Drive, Ottawa",
    "1125 Colonel By Dr, Ottawa",
    "54 Hawk Crescent, Ottawa",
    "1380 Prince of Wales Drive, Ottawa",
    "75 Laurier Ave. E, Ottawa, Ontario, Canada",
    "14 Waller St, Ottawa, Ontario, Canada",
    "363 Rideau St, Ottawa, Ontario, Canada",]

for addr in addrs:
    print(addr)
    coords.append(coords_from_address(addr))
    time.sleep(1)




2701 Riverside Dr, Ottawa
2515 Bank St, Ottawa
30 Haxby Private, Ottawa
116 Sai Crescent
214 Plymouth St, Ottawa
266 Lorry Greenberg Dr, Ottawa
800 King Edward Ave, Ottawa
2210 Bank St, Ottawa
2161 Thurston Dr, Ottawa
23 Ella Street, Ottawa
11 Oblats Avenue, Ottawa
422 Byron Avenue, Ottawa
1680 Fisher Avenue, Ottawa
825 Cahill Drive, Ottawa
1125 Colonel By Dr, Ottawa
54 Hawk Crescent, Ottawa
1380 Prince of Wales Drive, Ottawa
75 Laurier Ave. E, Ottawa, Ontario, Canada
14 Waller St, Ottawa, Ontario, Canada
363 Rideau St, Ottawa, Ontario, Canada


In [ ]:
#T_mat, D_mat = getGoogleTimeDistance(addrs)


In [36]:
#pickle
with open('T_mat','wb') as f1: 
    pickle.dump(T_mat, f1)

with open('T_mat','rb') as f2:
    new_T_mat = pickle.load(f2)
    
with open('D_mat','wb') as f3: 
    pickle.dump(D_mat, f3)

with open('D_mat','rb') as f4:
    new_D_mat = pickle.load(f4)

In [20]:
def assign_destinations(delivery_coordinates, addresses, n_trucks):
    """
    inputs: list(tuple) delivery coordinates, int n_trucks
    outputs: dict[list[int]] Allocation
    determines which trucks go where, referencing index of original address list and D_mat
    """
    coords_array = np.array(delivery_coords)
    kmeans = KMeans(n_clusters=n_trucks).fit(coords_array) 
    cluster_map = pd.DataFrame()
    #add one because the start location isnt counted
    cluster_map['destination_index'] = np.arange(len(coords_array))
    cluster_map['cluster'] = kmeans.labels_
    
    Allocation = {}
    for c_idx in set(cluster_map["cluster"]):
        location_details = []
        
        destination_idxs = list(cluster_map[cluster_map["cluster"]==c_idx]["destination_index"])
        for i in destination_idxs:
            
            loc_detail = (i+1, delivery_coordinates[i], addresses[i])
            
            location_details.append(loc_detail)
            
        Allocation[c_idx] = location_details
        #print(Allocation)
        
    return Allocation

def sub_DT_mats(Allocation):
    """
    input dict[list[int]] Allocation
    output:  dict[nparray] T_mats, dict[nparray] D_mats
    description: calculates distance and time matrices for allocation with distance and time matrices
    """
    T_mats = {}
    D_mats = {}
    for t in Allocation.keys():
        #start with canada post address
        addresses = ["2701 Riverside Dr, Ottawa"]
        for loc_detail in Allocation[t]:
            addresses.append(loc_detail[2])
        T_mat, D_mat = getGoogleTimeDistance(addresses)
        T_mats[t] = T_mat
        D_mats[t] = D_mat
        
    return T_mats, D_mats

def initial_configs(Allocation):
    result = copy.deepcopy(Allocation)
    
    
    for t in result.keys():
        for i in range(len(result[t])):
            result[t][i]=i+1
        result[t].insert(0,0)
    return result


In [32]:
start_configs[0][1]

1

In [35]:
allocated_destinations[0][1]

(12, (45.357181499999996, -75.70871728867277), '1680 Fisher Avenue, Ottawa')

In [21]:
start_configs = initial_configs(allocated_destinations)

In [13]:
#get delivery coordinates and addresses
delivery_coords = coords[1:]
delivery_addrs = addrs[1:]
n_trucks = 3
allocated_destinations = assign_destinations(delivery_coords, delivery_addrs, n_trucks)
start_configs = initial_configs(allocated_destinations)

T_mats, D_mats = sub_DT_mats(allocated_destinations)

In [17]:
T_mats

{0: array([[  0, 705, 388, 401, 260],
        [757,   0, 691, 911, 716],
        [335, 678,   0, 557, 252],
        [408, 881, 528,   0, 368],
        [281, 785, 307, 503,   0]]),
 1: array([[  0, 637, 606, 826, 686, 530, 589, 618, 614],
        [526,   0, 359, 494, 344, 307, 435, 409, 405],
        [661, 346,   0, 443, 184, 348, 279, 551, 547],
        [826, 472, 418,   0, 385, 608, 297, 709, 705],
        [599, 245, 176, 401,   0, 381, 237, 482, 478],
        [453, 308, 346, 641, 426,   0, 521, 335, 332],
        [756, 413, 275, 335, 243, 518,   0, 650, 646],
        [542, 375, 560, 709, 558, 403, 650,   0, 207],
        [561, 394, 578, 727, 577, 421, 668, 239,   0]]),
 2: array([[  0, 428, 857, 363, 489, 806, 950, 952],
        [486,   0, 769, 311, 547, 718, 767, 782],
        [596, 280,   0, 420, 224, 312, 541, 392],
        [406, 284, 716,   0, 495, 665, 765, 803],
        [600, 428, 405, 518,   0, 354, 583, 491],
        [723, 407, 127, 548, 351,   0, 301, 223],
        [810, 494

In [22]:
start_configs

{0: [0, 1, 2, 3, 4],
 1: [0, 1, 2, 3, 4, 5, 6, 7, 8],
 2: [0, 1, 2, 3, 4, 5, 6, 7]}

In [19]:
4+8+7

19

In [24]:
def calc_route_length(config, D_mat):
    
    route_length = 0
    #print(D_mat)
    #For each city, calculate the distance between it and previous.
    for i in range(len(config)):
        start_idx = config[i-1]
        end_idx = config[i]
        #print(start_idx,end_idx)
        
        d = D_mat[start_idx][end_idx]
        route_length = route_length + d
        
    return route_length

#Define a function to generate a trial configuration where we swap the order of visiting two cities.
def generate_trial_config(config):
    
    #Make a deep copy so we don't accidentally mess up our config:
    trial_config = deepcopy(config)
    
    #Generate one random city to switch:
    first_swap = np.random.randint(0, len(config))
    #Assign this value to our second swap:
    second_swap = first_swap
    
    #Generate a second random city to switch, making sure it's different from our first:
    while second_swap == first_swap:
        second_swap = np.random.randint(0, len(config))
    
    #Now swap our indices:
    hold_value = deepcopy(trial_config[first_swap])
    trial_config[first_swap] = trial_config[second_swap]
    trial_config[second_swap] = hold_value
    
    #print(trial_config)
    
    return trial_config, first_swap, second_swap

def calc_distance_change(config, trial_config, first_swap, second_swap, D_mat):
    
    #Calculate original distance:
    idx_i = config[first_swap]
    #print(idx_i)
    #Distance between first swap -1:
    idx_1_min = config[first_swap-1]
   
    #Pythagorean theorem:
    
    d_one_min = D_mat[idx_i][idx_1_min]
    #Distance between first swap +1, set our bounds on our array of values:
    if (first_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = first_swap +1
    idx_1_max = config[swap_max]
    
    #Pythagoran theorem:
    d_one_max = D_mat[idx_1_max][idx_i]
    #Distance between second swap -1:
    idx_j = config[second_swap]
    idx_2_min = config[second_swap-1]
    
    #Pythagorean theorem:
    
    d_two_min = D_mat[idx_j][idx_2_min]
    #Distance between second swap +1, set our bounds on array of values:
    if (second_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = second_swap +1
        
    idx_2_max = config[swap_max]
    
    #Pythagorean theorem:
    d_two_max = D_mat[idx_2_max][idx_j]
    
    #Now we have our original distance that navigating to these points took:
    original_distance = d_one_min + d_one_max + d_two_min + d_two_max
    #print(original_distance)
    
    #Now repeat for the trial configuration...
    #Calculate original distance:
    idx_i = trial_config[first_swap]
    
    #Distance between first swap -1:
    idx_1_min =  trial_config[first_swap-1]
   
    #Pythagorean theorem:
    d_one_min = D_mat[idx_i][idx_1_min]
    #Distance between first swap +1, set our bounds on our array of values:
    if (first_swap + 1) > (len(trial_config)-1):
        swap_max = 0
    else:
        swap_max = first_swap +1
    idx_1_max = trial_config[swap_max]
    
    #Pythagoran theorem:
    
    d_one_max = D_mat[idx_1_max][idx_i]
    
    #Distance between second swap -1:
    idx_j = trial_config[second_swap]
    idx_2_min = trial_config[second_swap-1]
    
    #Pythagorean theorem:
   
    d_two_min = D_mat[idx_j][idx_2_min]
    #Distance between second swap +1, set our bounds on array of values:
    if (second_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = second_swap +1
    idx_2_max = trial_config[swap_max]
    
    #Pythagorean theorem:
    d_two_max = D_mat[idx_2_max][idx_j]
    #Now we have our new distance that navigating to these points took:
    new_distance = d_one_min + d_one_max + d_two_min + d_two_max
    #print(new_distance)
    
    #Calculate our change in distance:
    delta_d = new_distance - original_distance
    #print(delta_d)
    
    return delta_d

#Define a function to calculate our probability and evaluate it:
def evaluate_route_change(delta_d, T):
    
    #Calculate our probability:
    P = math.exp((-1*delta_d)/(T))
    #Geneate uniform random variable:
    u = np.random.uniform()
    #Estimate whether we're keeping this or not:
    if u <= P:
        accept = True
    else:
        accept = False
    
    return accept


def minimize_route(config, T, roc_tol, mean_samples, D_mat):
    
    #Create counter for our number of steps, and initial large value for rate of change:
    numsteps = 0
    roc = 10*roc_tol
    
    #Find our initial route length - we'll need this later...
    route_length = calc_route_length(config, D_mat)
    #Create array to store new route lengths:
    route_lengths = []
    
    #Want rate of change of our config over some number of steps to be small...
    while roc > roc_tol:
        
        #Generate trial configuration and calulate change in distance:
        trial_config, first_swap, second_swap = generate_trial_config(config)
        delta_d = calc_distance_change(config, trial_config, first_swap, second_swap, D_mat)
        
        #Evaluate if we are keeping our new configuration or not...
        if delta_d <= 0:
            config = deepcopy(trial_config)
            route_length = route_length + delta_d
            route_lengths.append(route_length)
            #Increment numsteps counter:
            numsteps = numsteps +1
        else:
            accept = evaluate_route_change(delta_d, T)
            if accept == True:
                config = deepcopy(trial_config)
                route_length = route_length + delta_d
                route_lengths.append(route_length)
                #Increment numsteps counter:
                numsteps = numsteps +1
            else: 
                pass
        
        #If we have more than 100 steps, evaluate our mean route length over the last 50 steps,
        #compare to the 50 before, and see what our rate of change is.
        if numsteps > (mean_samples*2):
            new_mean = np.mean(route_lengths[numsteps-mean_samples:])
            old_mean = np.mean(route_lengths[numsteps-(mean_samples*2):numsteps-mean_samples])
            roc = abs(new_mean - old_mean)
            
    return config, route_lengths



def travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat):
    #print(D_mat)
    #Generate initial configuration:
    #initial_config = config_idx
    #print(initial_config)
    #Create equivalent of do-while loop:
    while True:
        #Do some math with our original route, including calculating route lengths etc.
        config = initial_config
        old_route_length = calc_route_length(config, D_mat)
        
        old_route_lengths = []
        old_route_lengths.append(old_route_length)
        #Create set of unique routes.
        unique_routes = set(np.around(old_route_lengths,3))
        #Set an initial roc we can evaluate against.
        roc = 10*roc_tol
        #Set our initial T.
        T = initial_T
        print(T)
        #Run while our roc is greater than our tol, and our temperature is above 0:
        while (roc > roc_tol) and (T > 0):
            #Minimize route at current T:
            config, route_lengths =  minimize_route(config, T, roc_tol, mean_samples, D_mat)
            #print(route_lengths)
            #If we've come up with a shorter route:
            if len(route_lengths) != 0:
                #Calculate change in mean:
                old_mean = np.mean(old_route_lengths)
                new_mean = np.mean(route_lengths)
                roc = abs(new_mean - old_mean)
                #If our roc is still greater:
                if roc > roc_tol:
                    #Swap over and update our recent routes...
                    old_route_lengths = deepcopy(route_lengths)
                    temp_set_routes = set(np.around(old_route_lengths,3))
                    unique_routes = unique_routes.union(temp_set_routes)
                else:
                    pass
            else:
                pass
            
            #Increment T downwards...
            T = T - delta_T
            print(T)
            
            #If T goes below zero...
            if T <= 0:
                route_lengths.append(old_route_lengths[-1])
            else:
                pass
        
        #Sort our set of routes so we can use these in failsafe.
        sorted_routes = sorted(unique_routes)
        #print(sorted_routes)
            
        #If we're not using the failsafe, or if our route is as short as the shortest possible route after hitting roc tolerance, break the loop.
        if (failsafe == False) or (round(route_lengths[-1],3) <= (round(sorted_routes[0],3)+0.001)):
            break
    
    return config, unique_routes, route_lengths[-1]


In [6]:
config0 = [(168, 15), (493, 72), (22, 299), (466, 75), (360, 263)]
config_idx = [0,1,2,3,4]
D_mat = np.zeros((len(config0), len(config0)))
for i in range(len(config0)):
    for k in range(len(config0)):
        if i !=k:
            D_mat[i][k] = np.sqrt((config0[i][0]-config0[k][0])**2+(config0[i][1]-config0[k][1])**2)

In [25]:
initial_T = 25
delta_T = 0.5
roc_tol_at_T = 0.1
roc_tol = 0.1
mean_samples = 100
failsafe = False
initial_config = start_configs[0]
D_mat = T_mats[0]

config, unique_routes, final_route_length = travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat)
print(config)
print(final_route_length)

25


KeyboardInterrupt: 

In [26]:
T_mats[0]

array([[  0, 705, 388, 401, 260],
       [757,   0, 691, 911, 716],
       [335, 678,   0, 557, 252],
       [408, 881, 528,   0, 368],
       [281, 785, 307, 503,   0]])

In [27]:
start_configs[0]

[0, 1, 2, 3, 4]

In [ ]:
minimize_route(config, T, roc_tol, mean_samples, D_mat)

In [25]:
def cost_function(start_idx, end_idx, distance_matrix, lookup_tables):
    """
    inputs: int start_idx, int end_idx, nparray distance_matrix, list[str] lookup_tables
    outputs: float cost
    description: calculates the cost to go from start_idx to end_idx. These idx correspond to distance matrix.
    cost also factors in weather and traffic, information could be obtained from a lookup table
    lookup_tables is filepath to csv file, which would be read to generate our lookup table
    """
    pass

def cost_matrix(distance_matrix, time_matrix, lookup_tables):
    """
    inputs: nparray distance_matrix, nparray time_matrix, str lookup_tables 
    outputs: nparray cost_mat
    description: returns an array that calculates the cost to go from one node to another, for all nodes
    """
    pass

def cluster_coords(coordinates, N_trucks):
    """
    inputs: list coordinates, int N_trucks
    outputs: list[list] grouped_coords
    description: groups the coordinates into n_clusters, where N_trucks = n_clusters
    grouped_coords is a list of lists containing coordinates, ex: grouped_coords[0] = list of all coordinates in cluster 0
    """
    pass

def sTSP(cost_matrix):
    """
    inputs: nparray cost_matrix
    outputs: list[tuple]
    description: calculates optimal sequence for a delivery truck
    """
    
def plot_route(travel_seq):
    """
    inputs: list[tuple] travel_seq
    outputs: none
    description: visualizes travel route on folium
    """